# Data Preparation

In [1]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        try:
            if col_type != object:
                
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
            else:
                #df[col] = df[col].astype('category')
                pass
        except:
            pass
            
    end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [2]:
# Basics
import pandas as pd
import numpy as np
import time
import os
from os import listdir
from os.path import isfile, join, basename
from tqdm import tqdm
from timeit import timeit
import gc

import math
import sys
import datetime
from dateutil.relativedelta import relativedelta

# Helper functions
#from lag_features import *
#from other_functions import *

# Preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [3]:
DIR = '/Users/carlosperezricardo/Desktop/TFM'
#DIR = '/content/drive/MyDrive/TFM'

AIRPORT = 10397
years_to_load = ['2017','2018','2019']

In [4]:
folder = os.path.join(DIR, 'datasets')
files = [f for f in listdir(folder) if isfile(join(folder, f))]
files = sorted(files)

In [5]:
df_path = os.path.join(DIR,'df_init.pkl')
if os.path.exists(df_path):
    df = pd.read_pickle(df_path)
else:
    st = time.time()

    df = pd.DataFrame()
    for file in files:
        loading = []
        if '.zip' in file:
            for year in years_to_load:
                loading.append(year in file)
            output = any(loading)
            if output:
                add_df = pd.read_csv(os.path.join(DIR,'datasets',file), parse_dates=['FL_DATE'])
                #add_df = add_df[(add_df.DEST_AIRPORT_ID == AIRPORT) | (add_df.ORIGIN_AIRPORT_ID == AIRPORT)]
                add_df = reduce_mem_usage(add_df)
                df = pd.concat([df, add_df], axis=0)
                del add_df
    df.reset_index(drop=True, inplace=True)

    et = time.time()
    print((et - st)/60)

    df = reduce_mem_usage(df)
    #df.to_pickle(df_path)

3.0323967297871905


In [6]:
df.head().transpose()

,0,1,2,3,4
FL_DATE,2017-01-01 00:00:00,2017-01-01 00:00:00,2017-01-01 00:00:00,2017-01-01 00:00:00,2017-01-01 00:00:00
OP_UNIQUE_CARRIER,AA,AA,AA,AA,AA
OP_CARRIER_AIRLINE_ID,19805,19805,19805,19805,19805
OP_CARRIER,AA,AA,AA,AA,AA
TAIL_NUM,N3GNAA,N3KAAA,N3LFAA,N3DDAA,N3HLAA
OP_CARRIER_FL_NUM,2186,2189,2190,2191,2192
ORIGIN_AIRPORT_ID,14100,11298,11298,11697,11278
ORIGIN_AIRPORT_SEQ_ID,1410002,1129804,1129804,1169704,1127803
ORIGIN_CITY_MARKET_ID,34100,30194,30194,32467,30852
ORIGIN,PHL,DFW,DFW,FLL,DCA


In [7]:
df = df[(df.CANCELLED != 1) & (df.DIVERTED != 1)]
df = df[(df.DEST_AIRPORT_ID == AIRPORT) | (df.ORIGIN_AIRPORT_ID == AIRPORT)]

# Feature Generation 

In [8]:
df['SCH_DEP_TIME'] = df['DEP_TIME'] - df['DEP_DELAY']
df['SCH_ARR_TIME'] = df['ARR_TIME'] - df['ARR_DELAY']

## Time features

In [9]:
def date_features(df, col):
    df[col] = pd.to_datetime(df[col])
    df['month'] = df[col].dt.month
    df['day'] = df[col].dt.day
    df['year'] = df[col].dt.year
    df['FL_DATE_quarter'] = df['FL_DATE'].dt.quarter
    df['weekday'] = df[col].dt.weekday
    df['year_month'] = [ str(y)+'_'+str(m) if m < 10 else str(y)+'_0'+str(m) for y, m in zip(df['year'], df['month']) ]
    df['year_week'] = df[col].dt.strftime('%Y%V')

    return df

In [10]:
df = date_features(df, 'FL_DATE')

## Flight features

In [11]:
airports_df = pd.read_csv(os.path.join(DIR,'airlines_airports', 'L_AIRPORT_ID.csv'))
airports_df = airports_df[['Code','Description','City_State','City','State_Country','lat','lon']]

In [12]:
# Use map rather than merge
lat_map = {key: value for (key, value) in zip(airports_df['Code'], airports_df['lat'])}
lon_map = {key: value for (key, value) in zip(airports_df['Code'], airports_df['lon'])}

In [13]:
df['lat_ORIGIN'] = df['ORIGIN_AIRPORT_ID'].map(lat_map)
df['lon_ORIGIN'] = df['ORIGIN_AIRPORT_ID'].map(lon_map)
df['lat_DEST'] = df['DEST_AIRPORT_ID'].map(lat_map)
df['lon_DEST'] = df['DEST_AIRPORT_ID'].map(lon_map)

In [14]:
def flight_distance(lat1, lon1, lat2, lon2):
    R = 6371
    phi1 = lat1 * math.pi/180
    phi2 = lat2 * math.pi/180
    delta_phi = (lat2-lat1) * math.pi/180
    delta_lambda = (lon2-lon1) * math.pi/180
    
    a = np.sin(delta_phi/2) * np.sin(delta_lambda/2) + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2) * np.sin(delta_lambda/2);
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    return np.round(R * c) # in kilometres

In [15]:
df['flight_distance'] = flight_distance(df['lat_ORIGIN'], df['lon_ORIGIN'], df['lat_DEST'], df['lon_DEST'])

/Users/carlosperezricardo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Weather features

In [16]:
WEATHER_DIR = os.path.join(DIR,'weather')
weather_df = pd.read_csv(os.path.join(WEATHER_DIR, 'weather_selected_2.csv'))

In [17]:
iata_code = list(df[df.DEST_AIRPORT_ID == AIRPORT]['DEST'].unique())[0]

airports_df = pd.read_csv(os.path.join(DIR,'airlines_airports', 'airports.csv'))

airports_code = pd.read_excel(os.path.join(DIR,'airlines_airports', 'US_airports.xlsx'), sheet_name='Airports')
airports_code.columns = airports_code.iloc[0]
airports_code = airports_code.iloc[1:]
    
states = pd.read_excel(os.path.join(DIR,'airlines_airports', 'US_airports.xlsx'), sheet_name='States')
states.columns = ['state_name','state_acronym']

airports = pd.merge(airports_code, states, left_on='City', right_on='state_name', how='left')

airports[['state_name','state_acronym']] = airports[['state_name','state_acronym']].ffill(axis = 0)
airports = airports[~airports['FAA'].isnull()]

In [18]:
icao_code = list(airports[airports.IATA == iata_code]['ICAO'])[0]
icao_code

'KATL'

In [19]:
weather_df = weather_df[weather_df.ICAO == icao_code]
weather_df['Day'] = pd.to_datetime(weather_df['Day'])

del airports, states, airports_df

weather_df.head()

,Unnamed: 0,Day,Temperature_Max,Temperature_Avg,Temperature_Min,Dew Point_Max,Dew Point_Avg,Dew Point_Min,Humidity_Max,Humidity_Avg,Humidity_Min,Wind Speed_Max,Wind Speed_Avg,Wind Speed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,Precipitation,ICAO
2192,2192,2016-01-01,46,39.7,35,29,26.7,25,72,53.8,42,13.0,10.1,6.0,29.4,29.3,29.2,0.00,KATL
2193,2193,2016-01-02,52,42.1,35,27,22.4,15,60,44.8,29,18.0,10.5,6.0,29.2,29.0,28.9,0.21,KATL
2194,2194,2016-01-03,44,38.3,32,17,14.2,12,60,43.3,29,16.0,9.9,5.0,28.9,28.9,28.9,0.00,KATL
2195,2195,2016-01-04,43,34.5,26,22,15.6,9,69,58.9,47,14.0,8.7,0.0,28.9,28.8,28.6,0.43,KATL
2196,2196,2016-01-05,44,36.6,26,45,35.0,23,93,87.0,80,14.0,9.1,0.0,29.0,28.8,28.6,0.00,KATL


In [20]:
df = pd.merge(df, weather_df, left_on='FL_DATE', right_on='Day', how='left')
del weather_df

In [21]:
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,Humidity_Avg,Humidity_Min,Wind Speed_Max,Wind Speed_Avg,Wind Speed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,Precipitation,ICAO
0,2017-01-01,AS,19930,AS,N423AS,746,14747,1474703,30559,SEA,...,74.3,46,13.0,5.8,0.0,29.0,28.9,28.8,0.48,KATL
1,2017-01-01,AS,19930,AS,N423AS,749,10397,1039705,30397,ATL,...,74.3,46,13.0,5.8,0.0,29.0,28.9,28.8,0.48,KATL
2,2017-01-01,AS,19930,AS,N448AS,750,14747,1474703,30559,SEA,...,74.3,46,13.0,5.8,0.0,29.0,28.9,28.8,0.48,KATL
3,2017-01-01,AS,19930,AS,N423AS,777,10397,1039705,30397,ATL,...,74.3,46,13.0,5.8,0.0,29.0,28.9,28.8,0.48,KATL
4,2017-01-01,AS,19930,AS,N464AS,901,10397,1039705,30397,ATL,...,74.3,46,13.0,5.8,0.0,29.0,28.9,28.8,0.48,KATL


## Operations features

In [22]:
# But first some cleaning
df = df[(df.CANCELLED != 1) & (df.DIVERTED != 1) & (~df.DEP_DELAY.isnull()) & (~df.ARR_DELAY.isnull())]

In [23]:
df['UPD_ARR_TIME'] = round((df['SCH_ARR_TIME'] + df['DEP_DELAY'])/100).astype(int)

df['SCH_DEP_TIME'] = round(df['SCH_DEP_TIME']/100).astype(int)
df['SCH_ARR_TIME'] = round(df['SCH_ARR_TIME']/100).astype(int)

### Scheduled

In [24]:
# Generate grid to perform lags properly
dates = sorted(list(df["FL_DATE"].unique()))
times = sorted(list(df["ARR_TIME"].unique()))

cartesian_product = pd.MultiIndex.from_product([dates, times], names = ["FL_DATE", "TIME"])
len(cartesian_product)

grid_df = pd.DataFrame(index = cartesian_product).reset_index()

dep = df[df.DEST_AIRPORT_ID == AIRPORT].groupby(['FL_DATE','SCH_ARR_TIME'])['OP_CARRIER_FL_NUM'].count().to_frame()
arr = df[df.ORIGIN_AIRPORT_ID == AIRPORT].groupby(['FL_DATE','SCH_DEP_TIME'])['OP_CARRIER_FL_NUM'].count().to_frame()

df_ = pd.merge(grid_df, dep, left_on=['FL_DATE','TIME'], right_on=['FL_DATE','SCH_ARR_TIME'], how = 'left')
df_ = pd.merge(df_, arr, left_on=['FL_DATE','TIME'], right_on=['FL_DATE','SCH_DEP_TIME'], how = 'left')

df_.fillna(0, inplace=True)
df_.columns = ['FL_DATE', 'TIME', 'airport_arr', 'airport_dep']

del grid_df, dates, times, dep, arr, cartesian_product

df_

,FL_DATE,TIME,airport_arr,airport_dep
0,2017-01-01,1.0,1.0,0.0
1,2017-01-01,2.0,0.0,2.0
2,2017-01-01,3.0,0.0,0.0
3,2017-01-01,4.0,1.0,0.0
4,2017-01-01,5.0,1.0,1.0
...,...,...,...,...
1476055,2019-12-31,2354.0,0.0,0.0
1476056,2019-12-31,2356.0,0.0,0.0
1476057,2019-12-31,2358.0,0.0,0.0
1476058,2019-12-31,2360.0,0.0,0.0


In [25]:
df_['airport_total_ops'] = df_['airport_arr'] + df_['airport_dep']

df_['airport_sum_ops'] = df_.groupby(['FL_DATE'])['airport_total_ops'].transform( lambda x: x.cumsum().shift(1))
df_['airport_sum_arr'] = df_.groupby(['FL_DATE'])['airport_arr'].transform( lambda x: x.cumsum().shift(1))
df_['airport_sum_dep'] = df_.groupby(['FL_DATE'])['airport_dep'].transform( lambda x: x.cumsum().shift(1))

# Last 2 hours
df_['airport_sum_ops_2h'] = df_.groupby(['FL_DATE'])['airport_total_ops'].transform( lambda x: x.rolling(2).sum().shift(1))

df_['airport_sum_arr_2h'] = df_.groupby(['FL_DATE'])['airport_arr'].transform( lambda x: x.rolling(2).sum().shift(1))
df_['airport_sum_dep_2h'] = df_.groupby(['FL_DATE'])['airport_dep'].transform( lambda x: x.rolling(2).sum().shift(1))

# Last 4 hours
df_['airport_sum_ops_4h'] = df_.groupby(['FL_DATE'])['airport_total_ops'].transform( lambda x: x.rolling(4).sum().shift(1))

df_['airport_sum_arr_4h'] = df_.groupby(['FL_DATE'])['airport_arr'].transform( lambda x: x.rolling(4).sum().shift(1))
df_['airport_sum_dep_4h'] = df_.groupby(['FL_DATE'])['airport_dep'].transform( lambda x: x.rolling(4).sum().shift(1))

df_.fillna(0, inplace=True)

int_cols = ['airport_arr','airport_dep','airport_total_ops','airport_sum_ops','airport_sum_arr','airport_sum_dep',\
            'airport_sum_ops_2h','airport_sum_arr_2h','airport_sum_dep_2h','airport_sum_ops_4h','airport_sum_arr_4h','airport_sum_dep_4h']
for col in int_cols:
    df_[col] = df_[col].astype(int)

df_

,FL_DATE,TIME,airport_arr,airport_dep,airport_total_ops,airport_sum_ops,airport_sum_arr,airport_sum_dep,airport_sum_ops_2h,airport_sum_arr_2h,airport_sum_dep_2h,airport_sum_ops_4h,airport_sum_arr_4h,airport_sum_dep_4h
0,2017-01-01,1.0,1,0,1,0,0,0,0,0,0,0,0,0
1,2017-01-01,2.0,0,2,2,1,1,0,0,0,0,0,0,0
2,2017-01-01,3.0,0,0,0,3,1,2,3,1,2,0,0,0
3,2017-01-01,4.0,1,0,1,3,1,2,2,0,2,0,0,0
4,2017-01-01,5.0,1,1,2,4,2,2,1,1,0,4,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476055,2019-12-31,2354.0,0,0,0,1551,775,776,0,0,0,0,0,0
1476056,2019-12-31,2356.0,0,0,0,1551,775,776,0,0,0,0,0,0
1476057,2019-12-31,2358.0,0,0,0,1551,775,776,0,0,0,0,0,0
1476058,2019-12-31,2360.0,0,0,0,1551,775,776,0,0,0,0,0,0


In [26]:
df['TIME'] = df['ARR_TIME']
df = pd.merge(df, df_, on=['FL_DATE','TIME'], how='left')
del df_

### Rescheduled

In [27]:
dates = sorted(list(df["FL_DATE"].unique()))
times = sorted(list(df["ARR_TIME"].unique()))

cartesian_product = pd.MultiIndex.from_product([dates, times], names = ["FL_DATE", "TIME"])
len(cartesian_product)

grid_df = pd.DataFrame(index = cartesian_product).reset_index()

dep = df[df.DEST_AIRPORT_ID == AIRPORT].groupby(['FL_DATE','UPD_ARR_TIME'])['OP_CARRIER_FL_NUM'].count().to_frame()
arr = df[df.ORIGIN_AIRPORT_ID == AIRPORT].groupby(['FL_DATE','SCH_DEP_TIME'])['OP_CARRIER_FL_NUM'].count().to_frame()

df_ = pd.merge(grid_df, dep, left_on=['FL_DATE','TIME'], right_on=['FL_DATE','UPD_ARR_TIME'], how = 'left')
df_ = pd.merge(df_, arr, left_on=['FL_DATE','TIME'], right_on=['FL_DATE','SCH_DEP_TIME'], how = 'left')

df_.fillna(0, inplace=True)
df_.columns = ['FL_DATE', 'TIME', 'update_airport_arr', 'airport_dep']

In [28]:
df_['update_airport_total_ops'] = df_['update_airport_arr'] + df_['airport_dep']

df_['update_airport_sum_ops'] = df_.groupby(['FL_DATE'])['update_airport_total_ops'].transform( lambda x: x.cumsum().shift(1))
df_['update_airport_sum_arr'] = df_.groupby(['FL_DATE'])['update_airport_arr'].transform( lambda x: x.cumsum().shift(1))

# Last 2 hours
df_['update_airport_sum_ops_2h'] = df_.groupby(['FL_DATE'])['update_airport_total_ops'].transform( lambda x: x.rolling(2).sum().shift(1))

df_['update_airport_sum_arr_2h'] = df_.groupby(['FL_DATE'])['update_airport_arr'].transform( lambda x: x.rolling(2).sum().shift(1))

df_.fillna(0, inplace=True)

int_cols = ['update_airport_arr','update_airport_total_ops','update_airport_sum_ops','update_airport_sum_arr',\
            'update_airport_sum_ops_2h','update_airport_sum_arr_2h']
drop_cols = ['airport_dep']
for col in int_cols:
    df_[col] = df_[col].astype(int)
for col in drop_cols:
    del df_[col]

df_.head()

,FL_DATE,TIME,update_airport_arr,update_airport_total_ops,update_airport_sum_ops,update_airport_sum_arr,update_airport_sum_ops_2h,update_airport_sum_arr_2h
0,2017-01-01,1.0,2,2,0,0,0,0
1,2017-01-01,2.0,0,2,2,2,0,0
2,2017-01-01,3.0,0,0,4,2,4,2
3,2017-01-01,4.0,1,1,4,2,2,0
4,2017-01-01,5.0,0,1,5,3,1,1


In [29]:
df = pd.merge(df, df_, on=['FL_DATE','TIME'], how='left')
del df_

In [30]:
# Select destination airport (to only consider arrivals)
df = df[df.DEST_AIRPORT_ID == AIRPORT]

In [31]:
# Outliers
print(df.shape)
df = df.loc[(df.ARR_DELAY >= -35) & (df.ARR_DELAY <= 150)]
print(df.shape)

(1135936, 83)
(1110432, 83)


## Summary and lag features

In [32]:
def q25(x):
    return x.quantile(0.25)


def q75(x):
    return x.quantile(0.75)

In [33]:
def calc_functions(fun):
    if 'mean':
        return np.mean
    elif 'median':
        return np.median
    elif 'min':
        return np.min
    elif 'max':
        return np.max
    elif 'std':
        return np.std
    elif 'q25':
        return q25
    elif 'q75':
        return q75
    else:
        print('No function calculation found')

In [34]:
def feature_generator(df, gb_list, target, fun):
    name_ =  [ target+'_' + '_'.join(gb_list)+'_'+fun]
    func = calc_functions(fun)
    df[name_] = df.groupby(gb_list)[target].transform(lambda x: func(x) )
    
    return(df)

In [35]:
# Origin and month
df = feature_generator(df, ['ORIGIN','month'], 'ARR_DELAY', 'mean')
df = feature_generator(df, ['ORIGIN','month'], 'ARR_DELAY', 'q25')
df = feature_generator(df, ['ORIGIN','month'], 'ARR_DELAY', 'q75')

# Origin state
df = feature_generator(df, ['ORIGIN_STATE_ABR','month'], 'ARR_DELAY', 'median')
df = feature_generator(df, ['ORIGIN_STATE_ABR','month'], 'ARR_DELAY', 'q25')
df = feature_generator(df, ['ORIGIN_STATE_ABR','month'], 'ARR_DELAY', 'q75')

In [36]:
# Carrier and Origin
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','ORIGIN'], 'ARR_DELAY', 'mean')
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','ORIGIN'], 'ARR_DELAY', 'q25')
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','ORIGIN'], 'ARR_DELAY', 'q75')

# Carrier, Origin and weekday
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','ORIGIN','weekday'], 'ARR_DELAY', 'median')
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','ORIGIN','weekday'], 'ARR_DELAY', 'q25')
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','ORIGIN','weekday'], 'ARR_DELAY', 'q75')

In [37]:
# Carrier, month and hour
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','month','SCH_ARR_TIME'], 'ARR_DELAY', 'mean')
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','month','SCH_ARR_TIME'], 'ARR_DELAY', 'q25')
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','month','SCH_ARR_TIME'], 'ARR_DELAY', 'q75')

df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','month','UPD_ARR_TIME'], 'ARR_DELAY', 'mean')
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','month','UPD_ARR_TIME'], 'ARR_DELAY', 'q25')
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','month','UPD_ARR_TIME'], 'ARR_DELAY', 'q75')

In [38]:
# year week (detail, could lead to data leakage in some cases)
df = feature_generator(df, ['OP_CARRIER_AIRLINE_ID','year_week'], 'ARR_DELAY', 'mean')
df = feature_generator(df, ['ORIGIN','year_week'], 'ARR_DELAY', 'mean')

df = feature_generator(df, ['year_week'], 'ARR_DELAY', 'mean')
df = feature_generator(df, ['year_week'], 'ARR_DELAY', 'median')
df = feature_generator(df, ['year_week'], 'ARR_DELAY', 'q25')
df = feature_generator(df, ['year_week'], 'ARR_DELAY', 'q75')

# Data Cleaning

In [39]:
df.shape

(1110432, 107)

## Missing values

In [40]:
missing_df = df.isnull().sum().to_frame()
missing_df.columns = ['nans']
missing_df[missing_df.nans != 0].sort_values('nans', ascending=False) / df.shape[0]*100

,nans
CANCELLATION_CODE,100.000000
Unnamed: 29,100.000000
CARRIER_DELAY,86.576035
WEATHER_DELAY,86.576035
NAS_DELAY,86.576035
LATE_AIRCRAFT_DELAY,86.576035
flight_distance,31.315560
Precipitation,19.818503
Pressure_Avg,16.478902
Pressure_Min,16.478902


In [41]:
drop_cols = ['CANCELLATION_CODE','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','LATE_AIRCRAFT_DELAY','Unnamed: 29','Unnamed: 0']

for col in drop_cols:
    try:
        del df[col]
    except:
        pass

In [42]:
missing_df = df.isnull().sum().to_frame()
missing_df.columns = ['nans']
missing_df[missing_df.nans != 0].sort_values('nans', ascending=False) / df.shape[0]*100

,nans
flight_distance,31.315560
Precipitation,19.818503
Pressure_Avg,16.478902
Pressure_Min,16.478902


### Imputing

In [43]:
# Flight distance
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(np.array(df['flight_distance']).reshape(-1, 1))
df['flight_distance'] = imp.transform(np.array(df['flight_distance']).reshape(-1, 1))

In [44]:
# Weather
weather_nans = ['Precipitation','Pressure_Avg','Pressure_Min']

for col in weather_nans:
    impute_map = df.groupby(['month'])[col].median().to_dict()

    df[col+'imputed'] = df['month'].map(impute_map)
    df.loc[df[col].isnull(), col] = df.loc[df[col].isnull(), col+'imputed']

    del df[col+'imputed']

In [45]:
missing_df = df.isnull().sum().to_frame()
missing_df.columns = ['nans']
missing_df[missing_df.nans != 0].sort_values('nans', ascending=False) / df.shape[0]*100

,nans


In [46]:
print(df.shape)

(1110432, 100)


# Feature selection

## Drop variables with high correlation

In [47]:
number_cols = list(df.select_dtypes(include=np.number).columns)
corr = df[number_cols].corr()
corr

,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEP_TIME,DEP_DELAY,...,ARR_DELAY_OP_CARRIER_AIRLINE_ID_month_SCH_ARR_TIME_q75,ARR_DELAY_OP_CARRIER_AIRLINE_ID_month_UPD_ARR_TIME_mean,ARR_DELAY_OP_CARRIER_AIRLINE_ID_month_UPD_ARR_TIME_q25,ARR_DELAY_OP_CARRIER_AIRLINE_ID_month_UPD_ARR_TIME_q75,ARR_DELAY_OP_CARRIER_AIRLINE_ID_year_week_mean,ARR_DELAY_ORIGIN_year_week_mean,ARR_DELAY_year_week_mean,ARR_DELAY_year_week_median,ARR_DELAY_year_week_q25,ARR_DELAY_year_week_q75
OP_CARRIER_AIRLINE_ID,1.000000,0.481049,-0.063294,-0.063294,0.012423,NaN,0.052013,NaN,0.017486,-0.007607,...,0.067819,0.062041,0.062041,0.062041,0.095606,0.054551,0.009742,0.009742,0.009742,0.009742
OP_CARRIER_FL_NUM,0.481049,1.000000,-0.052535,-0.052536,0.000656,NaN,0.024007,NaN,0.009768,0.013029,...,0.087912,0.081544,0.081544,0.081544,0.121320,0.042374,0.008265,0.008265,0.008265,0.008265
ORIGIN_AIRPORT_ID,-0.063294,-0.052535,1.000000,1.000000,0.677452,NaN,-0.009454,NaN,-0.007327,-0.015650,...,-0.014987,-0.015217,-0.015217,-0.015217,-0.032461,-0.057835,0.000762,0.000762,0.000762,0.000762
ORIGIN_AIRPORT_SEQ_ID,-0.063294,-0.052536,1.000000,1.000000,0.677453,NaN,-0.009452,NaN,-0.007327,-0.015650,...,-0.014987,-0.015217,-0.015217,-0.015217,-0.032461,-0.057835,0.000762,0.000762,0.000762,0.000762
ORIGIN_CITY_MARKET_ID,0.012423,0.000656,0.677452,0.677453,1.000000,NaN,-0.002350,NaN,-0.016147,-0.021630,...,-0.048189,-0.054329,-0.054329,-0.054329,-0.035909,-0.059706,0.001434,0.001434,0.001434,0.001434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ARR_DELAY_ORIGIN_year_week_mean,0.054551,0.042374,-0.057835,-0.057835,-0.059706,NaN,0.023020,NaN,0.035952,0.205423,...,0.178728,0.176404,0.176404,0.176404,0.619357,1.000000,0.613161,0.613161,0.613161,0.613161
ARR_DELAY_year_week_mean,0.009742,0.008265,0.000762,0.000762,0.001434,NaN,0.066383,NaN,0.007471,0.112938,...,0.143788,0.136220,0.136220,0.136220,0.775671,0.613161,1.000000,1.000000,1.000000,1.000000
ARR_DELAY_year_week_median,0.009742,0.008265,0.000762,0.000762,0.001434,NaN,0.066383,NaN,0.007471,0.112938,...,0.143788,0.136220,0.136220,0.136220,0.775671,0.613161,1.000000,1.000000,1.000000,1.000000
ARR_DELAY_year_week_q25,0.009742,0.008265,0.000762,0.000762,0.001434,NaN,0.066383,NaN,0.007471,0.112938,...,0.143788,0.136220,0.136220,0.136220,0.775671,0.613161,1.000000,1.000000,1.000000,1.000000


In [48]:
high_corr_df = pd.DataFrame()
visit_cols = list(df.select_dtypes(include=np.number).columns)

for x in df.select_dtypes(include=np.number).columns:
    for y in visit_cols:
        if x != y:
            if np.array(df[[x,y]].corr())[0,1] > 0.9:
                high_corr_df = high_corr_df.append({'var1':x, 'var2':y, 'corr':np.array(df[[x,y]].corr())[0,1]}, ignore_index=True)
                if x == 'ARR_DELAY' or y == 'ARR_DELAY':
                    print('WARNING!!!',x, ' and ', y ,' are highly correlated', np.array(df[[x,y]].corr())[0,1])
    visit_cols.remove(x)

In [49]:
high_corr_df.sort_values('corr',ascending=False).head(30)

,corr,var1,var2
113,1.000000,ARR_DELAY_year_week_q25,ARR_DELAY_year_week_q75
102,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_month_SCH_ARR_...,ARR_DELAY_OP_CARRIER_AIRLINE_ID_month_SCH_ARR_...
92,1.000000,ARR_DELAY_ORIGIN_month_q25,ARR_DELAY_ORIGIN_month_q75
93,1.000000,ARR_DELAY_ORIGIN_STATE_ABR_month_median,ARR_DELAY_ORIGIN_STATE_ABR_month_q25
94,1.000000,ARR_DELAY_ORIGIN_STATE_ABR_month_median,ARR_DELAY_ORIGIN_STATE_ABR_month_q75
95,1.000000,ARR_DELAY_ORIGIN_STATE_ABR_month_q25,ARR_DELAY_ORIGIN_STATE_ABR_month_q75
96,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_mean,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_q25
97,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_mean,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_q75
98,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_q25,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_q75
99,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_weekday...,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_weekday...


In [50]:
high_corr_df.sort_values('corr',ascending=False).head(30)

,corr,var1,var2
113,1.000000,ARR_DELAY_year_week_q25,ARR_DELAY_year_week_q75
102,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_month_SCH_ARR_...,ARR_DELAY_OP_CARRIER_AIRLINE_ID_month_SCH_ARR_...
92,1.000000,ARR_DELAY_ORIGIN_month_q25,ARR_DELAY_ORIGIN_month_q75
93,1.000000,ARR_DELAY_ORIGIN_STATE_ABR_month_median,ARR_DELAY_ORIGIN_STATE_ABR_month_q25
94,1.000000,ARR_DELAY_ORIGIN_STATE_ABR_month_median,ARR_DELAY_ORIGIN_STATE_ABR_month_q75
95,1.000000,ARR_DELAY_ORIGIN_STATE_ABR_month_q25,ARR_DELAY_ORIGIN_STATE_ABR_month_q75
96,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_mean,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_q25
97,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_mean,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_q75
98,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_q25,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_q75
99,1.000000,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_weekday...,ARR_DELAY_OP_CARRIER_AIRLINE_ID_ORIGIN_weekday...


In [66]:
to_drop = []
for key, row in high_corr_df.iterrows():
    #print(row['var1'], row['var2'], row['corr'])
    if row['corr'] > 0.99:
        try:
            to_drop.append(row['var2'])
        except: 
            pass

In [67]:
to_drop

['ORIGIN_AIRPORT_SEQ_ID',
 'SCH_DEP_TIME',
 'SCH_ARR_TIME',
 'UPD_ARR_TIME',
 'TIME',
 'UPD_ARR_TIME',
 'TIME',
 'TIME',
 'update_airport_arr',
 'airport_total_ops',
 'update_airport_total_ops',
 'update_airport_total_ops',
 'airport_sum_arr',
 'airport_sum_dep',
 'update_airport_sum_ops',
 'update_airport_sum_arr',
 'airport_sum_dep',
 'update_airport_sum_ops',
 'update_airport_sum_arr',
 'update_airport_sum_ops',
 'update_airport_sum_arr',
 'airport_sum_arr_2h',
 'airport_sum_dep_2h',
 'update_airport_sum_ops_2h',
 'update_airport_sum_arr_2h',
 'update_airport_sum_ops_2h',
 'update_airport_sum_arr_2h',
 'update_airport_sum_ops_2h',
 'airport_sum_arr_4h',
 'airport_sum_dep_4h',
 'airport_sum_dep_4h',
 'update_airport_total_ops',
 'update_airport_sum_arr',
 'update_airport_sum_arr_2h',
 'ARR_DELAY_ORIGIN_month_q25',
 'ARR_DELAY_ORIGIN_month_q75',
 'ARR_DELAY_ORIGIN_month_q75',
 'ARR_DELAY_ORIGIN_STATE_ABR_month_q25',
 'ARR_DELAY_ORIGIN_STATE_ABR_month_q75',
 'ARR_DELAY_ORIGIN_STATE_ABR

In [68]:
len(to_drop)

58

In [69]:
for col in to_drop:
    try:
        del df[col]
    except:
        pass

In [70]:
df.shape

(1110432, 67)

In [76]:
df.replace({np.inf:np.nan}, inplace=True)
df.replace({-np.inf:np.nan}, inplace=True)
df.replace({-np.nan:np.nan}, inplace=True)

df.fillna(0, inplace=True)

In [ ]:
df = reduce_mem_usage(df)

In [ ]:
df.to_pickle(os.path.join(DIR, 'df_preprocessed.pkl'))